In [1]:
import os
import re
import pandas as pd
import numpy as np

In [2]:
column_dict = {
    'city':'city',
    'yearmonth':'yearmonth',
    '鄉鎮市區':'district',
'交易標的':'trade_target',
'土地區段位置建物區段門牌':'address',
'土地移轉總面積平方公尺':'land_area',
'都市土地使用分區':'area_use',
'非都市土地使用分區':'nonmetro_d',
'非都市土地使用編定':'nonmetro_u',
'交易年月日':'trade_date',
'交易筆棟數':'target_detail',
'移轉層次':'trade_floor',
'總樓層數':'total_floor',
'建物型態':'state',
'主要用途':'purpose',
'主要建材':'materials',
'建築完成年月':'building_date',
'建物移轉總面積平方公尺':'building_area',
'建物現況格局-房':'room',
'建物現況格局-廳':'hall',
'建物現況格局-衛':'health',
'建物現況格局-隔間':'compartment',
'有無管理組織':'manage',
'總價元':'price',
'單價元平方公尺':'unit_price',
'車位類別':'berth_type',
'車位移轉總面積平方公尺':'berth_area',
'車位總價元':'berth_price',
'備註':'note',
'編號':'trade_id'}
column_dict_inv = {v: k for k, v in column_dict.items()}

In [3]:
df = pd.read_csv('../original/107_1/A_lvr_land_A.csv')
df.drop([0], inplace=True)
col = list(df.columns)
col.insert(0, 'city')
merge_df = pd.DataFrame(columns=col)
print(merge_df)

Empty DataFrame
Columns: [city, 鄉鎮市區, 交易標的, 土地區段位置建物區段門牌, 土地移轉總面積平方公尺, 都市土地使用分區, 非都市土地使用分區, 非都市土地使用編定, 交易年月日, 交易筆棟數, 移轉層次, 總樓層數, 建物型態, 主要用途, 主要建材, 建築完成年月, 建物移轉總面積平方公尺, 建物現況格局-房, 建物現況格局-廳, 建物現況格局-衛, 建物現況格局-隔間, 有無管理組織, 總價元, 單價元平方公尺, 車位類別, 車位移轉總面積平方公尺, 車位總價元, 備註, 編號]
Index: []

[0 rows x 29 columns]


In [4]:
city_dict = {
    'A':'臺北市', 'J':'新竹縣', 'S':'高雄縣', 
    'B':'臺中市', 'K':'苗栗縣', 'T':'屏東縣', 
    'C':'基隆市', 'L':'臺中縣', 'U':'花蓮縣', 
    'D':'臺南市', 'M':'南投縣', 'V':'臺東縣', 
    'E':'高雄市', 'N':'彰化縣', 'W':'金門縣', 
    'F':'臺北縣', 'O':'新竹市', 'X':'澎湖縣', 
    'G':'宜蘭縣', 'P':'雲林縣', 'Y':'陽明山', 
    'H':'桃園縣', 'Q':'嘉義縣', 'Z':'連江縣', 
    'I':'嘉義市', 'R':'臺南縣'}

In [5]:
for foldername in os.listdir("../original"):
    if re.match("10*", foldername):
        i = 0
        for filename in os.listdir("../original/" + foldername):
            if re.match("[A-Z]_lvr_land_*", filename):
                i+=1
#                 print('../original/'+ foldername + '/' + filename)
                tmp_df = pd.read_csv('../original/'+ foldername + '/' + filename)
                tmp_df.drop([0], inplace=True)
#                 print(merge_df.shape, tmp_df.shape)
                tmp_df['city'] = city_dict[filename[0]]
                merge_df = merge_df.append(tmp_df, ignore_index=True)
        print(foldername, i)

107_1 22
107_2 22
107_3 22
107_4 22
108_1 22
108_2 22
108_3 22
108_4 22
109_1 22


In [6]:
print(merge_df.shape)
merge_df = merge_df.loc[[len(x) >= 6 for x in merge_df['交易年月日']]]
merge_df.reset_index(drop=True, inplace=True)
merge_df = merge_df[merge_df['鄉鎮市區'].notna()]
merge_df.reset_index(drop=True, inplace=True)
print(merge_df.shape)
merge_df['yearmonth'] = [ x[:-2] for x in merge_df['交易年月日']]
merge_df.rename(columns=column_dict, inplace=True)
print(merge_df[:3])

(724394, 29)
(724391, 29)
  city district  trade_target              address land_area area_use  \
0  臺北市      文山區     房地(土地+建物)   臺北市文山區景興路202巷1~30號       7.8        商   
1  臺北市      文山區  房地(土地+建物)+車位  臺北市文山區木柵路三段151~180號     16.37        商   
2  臺北市      文山區     房地(土地+建物)  臺北市文山區景福街54巷5弄1~30號     27.52        住   

  nonmetro_d nonmetro_u trade_date target_detail  ... compartment manage  \
0        NaN        NaN    1060926     土地1建物1車位0  ...           有      有   
1        NaN        NaN    1060804     土地1建物1車位1  ...           有      有   
2        NaN        NaN    1061010     土地1建物1車位0  ...           有      有   

      price unit_price berth_type berth_area berth_price            note  \
0  10200000     151515        NaN        0.0           0             NaN   
1  42400000     188917       坡道平面      42.94     2000000  102.4.3預售屋買賣過戶   
2  20300000     163486        NaN        0.0           0             NaN   

              trade_id yearmonth  
0  RPQOMLPKJIMFFAA37CA     10609  
1

In [8]:
# decompose
land_df = merge_df[['nonmetro_d', 'nonmetro_u', 'trade_id']].copy()
land_df.dropna(thresh=2, inplace=True)
land_df.reset_index(drop=True, inplace=True)
land_df.to_csv('2_merge_land_info.csv', index=False, na_rep='NULL')
print(land_df.shape)

(133679, 3)


In [9]:
struct_df = merge_df[['trade_floor','total_floor','purpose','materials','building_date','trade_id']].copy()
struct_df.dropna(thresh=5, inplace=True)
struct_df.reset_index(drop=True, inplace=True)
struct_df.to_csv('2_merge_struct_info.csv', index=False, na_rep='NULL')
print(struct_df.shape)

(534657, 6)


In [10]:
others_df = merge_df[['berth_type','note','trade_id']].copy()
others_df.dropna(thresh=2, inplace=True)
others_df.reset_index(drop=True, inplace=True)
others_df.to_csv('2_merge_others_info.csv', index=False, na_rep='NULL')
print(others_df.shape)

(407549, 3)


In [14]:
small_merge = merge_df.drop(['nonmetro_d', 'nonmetro_u',\
               'trade_floor', 'total_floor', 'purpose', 'materials', 'building_date',\
               'berth_type', 'note'], axis=1)
small_merge.to_csv('2_merge.csv', index=False, na_rep='NULL')
print(small_merge.shape)
print(small_merge.columns)

(724391, 21)
Index(['city', 'district', 'trade_target', 'address', 'land_area', 'area_use',
       'trade_date', 'target_detail', 'state', 'building_area', 'room', 'hall',
       'health', 'compartment', 'manage', 'price', 'unit_price', 'berth_area',
       'berth_price', 'trade_id', 'yearmonth'],
      dtype='object')


In [13]:
tmp_df = pd.DataFrame([ x[:-4] for x in merge_df['trade_date']])
tmp_df[0].value_counts()

108    300667
107    298939
106     92593
105     11071
104      7068
103      4598
109      4214
102      2948
101      2019
100        81
99         61
96         20
98         19
97          9
92          8
80          7
93          6
95          6
91          5
18          4
84          4
86          4
70          4
65          3
66          3
78          3
94          3
88          2
85          2
62          2
87          2
64          2
63          1
69          1
76          1
82          1
77          1
89          1
75          1
17          1
15          1
14          1
79          1
71          1
81          1
56          1
Name: 0, dtype: int64